In [1]:
#imports
import os, warnings
import matplotlib.pyplot as plt
from matplotlib import gridspec
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [8]:
# Load training and validation sets
ds_train_ = image_dataset_from_directory(
    '../First-Gen_Pokemon_Image_Classifier/dataset/train/Mewtwo',
    labels= None,
    label_mode='binary',
    image_size=[128, 128],
    interpolation='nearest',
    batch_size=64,
    shuffle=True,
)

Found 26 files.
